In [120]:
import numpy as np
import pandas as pd
from pathlib import Path
import mist_cf.common as common
import matplotlib.pyplot as plt
import copy

In [121]:
# 3 necessary files:
# CASMI_2022_key.csv (https://docs.google.com/spreadsheets/d/1xEblHpuz6zJh5Sbbm8q1QhWQGpCstFSj/edit#gid=736896340)
# CASMI_quant.csv (export from mzmine)
# CASMI.mgf (export from mzmine)

In [122]:
mgf_file = "../../data/casmi22/CASMI.mgf"
quant_file = "../../data/casmi22/CASMI_quant.csv"
casmi_file = "../../data/casmi22/CASMI_2022_key.csv"

export_mgf = "../../data/casmi22/CASMI_processed.mgf"
export_casmi = "../../data/casmi22/CASMI_2022_key_processed.csv"
export_labels = "../../data/casmi22/CASMI_labels.tsv"

In [123]:
quant_df = pd.read_csv(quant_file)
quant_df = quant_df[quant_df["row m/z"].notna()]
quant_df = quant_df[quant_df["row retention time"].notna()]

col_names = [i for i in quant_df.columns if "Peak area" in i]

# Melt df and keep columns 'row ID', 'row m/z', 'row retention time',
# Keep cols with col_names
new_df = pd.melt(
    quant_df,
    id_vars=["row ID", "row m/z", "row retention time"],
    value_vars=col_names,
    var_name="Sample",
    value_name="Peak area",
)
new_df["Peak area"] = new_df["Peak area"].astype(float)

# Greater than 0 filter for peak area
new_df = new_df[new_df["Peak area"] > 0]
# Sort by row ID
new_df = new_df.sort_values(by=["row ID"])
quant_df = new_df
quant_df["File"] = quant_df["Sample"].apply(lambda x: x.replace(".mzml Peak area", ""))
quant_df.reset_index(drop=True, inplace=True)
quant_df

,row ID,row m/z,row retention time,Sample,Peak area,File
0,109,117.065640,0.223683,E_M76_posPFP_01.mzml Peak area,40447.574,E_M76_posPFP_01
1,189,747.849250,0.228936,E_M33_posPFP_01.mzml Peak area,593956.700,E_M33_posPFP_01
2,189,747.849250,0.228936,E_M33_posPFP_02.mzml Peak area,487141.720,E_M33_posPFP_02
3,190,777.859695,0.228936,E_M33_posPFP_02.mzml Peak area,1063560.000,E_M33_posPFP_02
4,190,777.859695,0.228936,E_M33_posPFP_01.mzml Peak area,1313050.900,E_M33_posPFP_01
...,...,...,...,...,...,...
5652,41536,185.983131,8.613734,E_M41_posPFP_02.mzml Peak area,11968.761,E_M41_posPFP_02
5653,41545,132.031877,8.614662,E_M27_posPFP_01.mzml Peak area,8440.904,E_M27_posPFP_01
5654,41545,132.031877,8.614662,E_M44_posPFP_01.mzml Peak area,10273.945,E_M44_posPFP_01
5655,41545,132.031877,8.614662,E_M27_posPFP_02.mzml Peak area,5298.693,E_M27_posPFP_02


In [124]:
casmi_df = pd.read_csv(casmi_file, header=0)
casmi_df = casmi_df[casmi_df["Compound Number"].notna()]

# Filter so that "File" must contain "pos"
casmi_df = casmi_df[casmi_df["File"].str.contains("pos")].reset_index(drop=True)
casmi_df

,Compound Number,File,RT [min],Precursor m/z (Da),Priority/Bonus,Adduct,InChIKey,SMILES,Collection,Rack ID or UCD Barcode,Well ID,Formula,Monoisotopic Mass,Mix or Set Number
0,3,A_M3_posPFP_01,5.55,719.2546,Priority,[M+H]+,UYALDZZEAZIEME-MYTBKXFISA-N,CC1C(C(C(C(O1)OC2=C(OC3=C(C(=CC(=C3C2=O)O)O)CC...,Analyticon,AD205398-03,E02,C35H42O16,718.2473,M-3
1,4,A_M6_posPFP_02,5.74,467.2728,Priority,[M+Na]+,IFQVIMQHTPDZQU-UHFFFAOYSA-N,CC(C)CC(C(=O)O)NC(=O)C(C(C)C)OC(=O)C(C(C)C)NC(...,Analyticon,AD205398-03,F07,C22H40N2O7,444.2836,M-6
2,5,A_M23_posPFP_02,5.55,499.2302,Priority,[M+Na]+,XPXRFYSHQGUVFK-UTXMSIPTSA-N,CC1CC2(C(C1O)C=C(C(CC3C(C3(C)C)C=C(C2=O)C)OC(=...,Analyticon,AD205398-06,G03,C26H36O8,476.2410,M-23
3,6,A_M26_posPFP_01,5.23,1102.5793,Priority,[M+NH4]+,XPCKJVBBSBPFQL-DCULSWDLSA-N,CC1C(C(C(C(O1)OC2C(OC(C(C2O)O)OCC3C(C(C(C(O3)O...,Analyticon,AD205398-02,F01,C54H84O22,1084.5454,M-26
4,9,E_M28_posPFP_01,3.41,336.1740,Priority,[M+H]+,WKEMJKQOLOHJLZ-UHFFFAOYSA-N,CN(C)CCC1=CNC2=C1C=C(C=C2)CS(=O)(=O)N3CCCC3,Emolcules,139759214,--,C17H25N3O2S,335.1667,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,492,A_M12_posPFP_01,3.65,430.2435,Bonus,[M+NH4]+,AKZZXWNFVQXXFN-DVMTYXMPSA-N,CC1CCC(C2(C1=CC(CC2)C(=C)C(=O)O)C)OC3C(C(C(C(O...,Analyticon,AD205398-02,B02,C21H32O8,412.2097,M-12
300,493,A_M14_posPFP_01,6.02,478.2435,Bonus,[M+NH4]+,KDVYEAVKWMGOPV-CMJOXMDJSA-N,CC=CC1=CC(=C(C(=C1)OC)OC(C)C(C2=CC(=C(C(=C2)OC...,Analyticon,AD205398-03,A04,C25H32O8,460.2097,M-14
301,494,A_M7_posPFP_01,1.63,358.1496,Bonus,[M+NH4]+,KPYQJVYNSWDFQU-TXUJKZCJSA-N,COC(=O)C=CC1=CC=C(C=C1)OC2C(C(C(C(O2)CO)O)O)O,Analyticon,AD205398-03,H07,C16H20O8,340.1158,M-7
302,495,A_M19_posPFP_01,1.77,578.2079,Bonus,[M+NH4]+,ZDGWEUBHJUPSEX-ASDNCSIPSA-N,COC1=C2C(=CC(=C1OC3C(C(C(C(O3)CO)O)O)OC4C(C(C(...,Analyticon,AD205398-02,G03,C24H32O15,560.1741,M-19


In [125]:
# No missing feats, great!
casmi_files = casmi_df["File"].values
target_files = quant_df["File"].values
match_files = casmi_files[:, None] == target_files[None, :]

casmi_rts = casmi_df["RT [min]"].values.astype("float")
target_rts = quant_df["row retention time"].values.astype("float")
match_rts = np.abs(casmi_rts[:, None] - target_rts[None, :]) < 0.1

casmi_mzs = casmi_df["Precursor m/z (Da)"].values.astype("float")
target_mzs = quant_df["row m/z"].values.astype("float")
match_mzs = np.abs(casmi_mzs[:, None] - target_mzs[None, :]) < 0.005

select = np.logical_and(match_rts, match_mzs)
select = np.logical_and(select, match_files)
missing_feat = casmi_df[~(select.sum(-1) > 0)]
display(missing_feat)
# select.sum(-1)

,Compound Number,File,RT [min],Precursor m/z (Da),Priority/Bonus,Adduct,InChIKey,SMILES,Collection,Rack ID or UCD Barcode,Well ID,Formula,Monoisotopic Mass,Mix or Set Number


In [126]:
rt_score = np.abs(casmi_rts[:, None] - target_rts[None, :])
mass_score = np.abs(casmi_mzs[:, None] - target_mzs[None, :])

ind_sel_score = mass_score + rt_score
ind_sel_score[~select] = 99999999
best_inds = np.argmin(ind_sel_score, -1)
x = np.arange(len(best_inds))
rowIDs = quant_df.loc[best_inds]["row ID"]
# ind_sel_score.shape, quant_df.shape
mgf_inds = rowIDs.values

In [127]:
casmi_df["rowIDs"] = mgf_inds

## Construct mgf output

In [128]:
# Read in mgf
parsed_mgf = common.parse_spectra_mgf(mgf_file)

3286it [00:00, 17077.61it/s]


In [129]:
len(parsed_mgf)

1643

In [130]:
feat_id_to_entry = {i[0]["FEATURE_ID"]: i for i in parsed_mgf}

In [131]:
# feat_id_to_entry['109']
casmi_df["rowIDs"].values[:10]

array([24770, 28403, 24914, 20342, 11826,  8129, 34216, 17815, 14547,
        7617])

In [132]:
keep_entries = []
ctr = 0
for _, casmi_df_row in casmi_df.iterrows():
    feat_id = str(casmi_df_row["rowIDs"])
    casmi_id = casmi_df_row["Compound Number"]

    if feat_id in feat_id_to_entry:
        sub_entry = copy.deepcopy(feat_id_to_entry[feat_id])
        sub_entry[0]["CASMI_ID"] = casmi_id
        if casmi_id == 211:
            print(casmi_id, sub_entry[0]["CASMI_ID"])
        keep_entries.append(sub_entry)
    else:
        print(f"Missing feature ID: {feat_id}")
        ctr += 1

211 211


In [133]:
# Find where compound number == 211
casmi_df[casmi_df["Compound Number"] == 211]

,Compound Number,File,RT [min],Precursor m/z (Da),Priority/Bonus,Adduct,InChIKey,SMILES,Collection,Rack ID or UCD Barcode,Well ID,Formula,Monoisotopic Mass,Mix or Set Number,rowIDs
110,211,A_M30_posPFP_01,6.02,554.2748,Priority,[M+NH4]+,BYTPMMJRDFCGKX-RYXVVOBLSA-N,CC1C(C(OC1C2=CC3=C(C=C2)OCO3)C4=CC(=C(C=C4)OC(...,Analyticon,AD205398-02,B09,C31H36O8,536.241,M-30,34314


In [134]:
feat_ids = set([i[0]["FEATURE_ID"] for i in keep_entries])
casmi_ids = set([i[0]["CASMI_ID"] for i in keep_entries])
print(f"Number of features: {len(feat_ids)}")
print(f"Number of CASMI IDs: {len(casmi_ids)}")

Number of features: 295
Number of CASMI IDs: 304


In [135]:
for keep_entry in keep_entries:
    old_feat_id = keep_entry[0]["FEATURE_ID"]
    new_feat_id = keep_entry[0]["CASMI_ID"]
    keep_entry[0]["OLD_FEAT_ID"] = old_feat_id
    keep_entry[0]["FEATURE_ID"] = new_feat_id

In [136]:
# Export casmi_key
casmi_df.to_csv(export_casmi, index=False)

In [137]:
# dataset	spec	name	formula	ionization	smiles	inchikey	instrument
new_df = []
for _, row in casmi_df.iterrows():
    new_entry = {
        "dataset": "casmi22",
        "spec": row["Compound Number"],
        "name": row["Priority/Bonus"],
        "formula": row["Formula"],
        "ionization": row["Adduct"],
        "smiles": row["SMILES"],
        "inchikey": row["InChIKey"],
        "instrument": "Orbitrap (LCMS)",
    }
    new_df.append(new_entry)
out_df = pd.DataFrame(new_df);
# casmi_df

In [138]:
out_df.to_csv(export_labels, sep="\t", index=False)

In [139]:
print(len(keep_entries))
# Get unique spec
unique_spec = set([i[0]["FEATURE_ID"] for i in keep_entries])
unique_spec_labels = set(out_df["spec"].values)
# Check set difference

print(len(unique_spec), len(unique_spec_labels))
# Print difference
print(unique_spec - unique_spec_labels)
print(unique_spec_labels - unique_spec)

304
304 304
set()
set()


In [140]:
len(keep_entries)

304

In [141]:
#
[i for i in keep_entries if int(i[0]["FEATURE_ID"]) == 211]

[({'FEATURE_ID': 211,
   'PEPMASS': '554.2743',
   'SCANS': '34314',
   'RTINSECONDS': '361.657',
   'CHARGE': '1+',
   'MSLEVEL': '2',
   'MERGED_STATS': '2 / 12 (0 removed due to low quality, 10 removed due to low cosine).',
   'CASMI_ID': 211,
   'OLD_FEAT_ID': '34314'},
  [('spec',
    array([[5.024870e+01, 2.300000e+03],
           [5.295700e+01, 2.500000e+03],
           [5.904930e+01, 4.500000e+03],
           [6.333790e+01, 2.600000e+03],
           [6.517620e+01, 2.600000e+03],
           [8.304920e+01, 3.000000e+03],
           [8.766820e+01, 3.000000e+03],
           [9.858280e+01, 2.400000e+03],
           [1.070493e+02, 4.500000e+03],
           [1.110935e+02, 2.500000e+03],
           [1.122068e+02, 2.400000e+03],
           [1.163162e+02, 2.400000e+03],
           [1.310493e+02, 3.400000e+03],
           [1.350441e+02, 2.300000e+05],
           [1.430853e+02, 6.500000e+03],
           [1.450648e+02, 7.600000e+04],
           [1.470804e+02, 4.700000e+04],
           [1.49

In [142]:
# Export mgf
out_str = common.build_mgf_str(keep_entries)
with open(export_mgf, "w") as f:
    f.write(out_str)

100%|██████████| 304/304 [00:00<00:00, 4108.31it/s]
